In [46]:
import pandas as pd
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# Load the dataset
df = pd.read_csv('PERC_mendelly.csv')

# Divide the poem into stanzas
df['Stanza'] = df['Poem'].apply(lambda x: x.split('\n\n'))

# Create a new dataframe to hold the emotion for each line in the stanza
stanza_df = pd.DataFrame(columns=['Stanza', 'Emotion'])

# Loop through each stanza
for i in range(len(df)):
    poem = df.iloc[i]['Poem']
    stanzas = df.iloc[i]['Stanza']
    
    # Loop through each line in the stanza
    for stanza in stanzas:
        lines = nltk.sent_tokenize(stanza)
        for line in lines:
            stanza_df = stanza_df.append({'Stanza': line, 'Emotion': df.iloc[i]['Emotion']}, ignore_index=True)

# Convert the text data to numerical form
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(stanza_df['Stanza'])

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, stanza_df['Emotion'], test_size=0.2)

# Train the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Test the classifier
y_pred = clf.predict(X_test)

# Evaluate the performance of the classifier
from sklearn.metrics import accuracy_score, classification_report
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test, y_pred))
# Define a function to predict the emotion for each stanza in a new poem
def predict_emotion(poem):
    stanzas = poem.split('\n\n')
    stanza_df = pd.DataFrame(columns=['Stanza'])
    for stanza in stanzas:
        lines = nltk.sent_tokenize(stanza)
        for line in lines:
            stanza_df = stanza_df.append({'Stanza': line}, ignore_index=True)
    X = vectorizer.transform(stanza_df['Stanza'])
    y_pred = clf.predict(X)
    stanza_df['Emotion'] = y_pred
    return stanza_df

# Call the predict_emotion function with a new poem
new_poem = """
My mother bore me in the southern wild,
And I am black, but O!my soul is white;
White as an angel is the English child: 
But I am black as if bereav'd of light.

My mother taught me underneath a tree 
And sitting down before the heat of day,
She took me on her lap and kissed me,
And pointing to the east began to say. 

Look on the rising sun: there God does live 
And gives his light, and gives his heat away. 
And flowers and trees and beasts and men receive
Comfort in morning joy in the noonday.

And we are put on earth a little space,
That we may learn to bear the beams of love, 
And these black bodies and this sun-burnt face
Is but a cloud, and like a shady grove.

For when our souls have learn'd the heat to bear 
The cloud will vanish we shall hear his voice. 
Saying: come out from the grove my love & care,
And round my golden tent like lambs rejoice.

Thus did my mother say and kissed me, 
And thus I say to little English boy. 
When I from black and he from white cloud free,
And round the tent of God like lambs we joy: 

Ill shade him from the heat till he can bear, 
To lean in joy upon our fathers knee. 
And then I'll stand and stroke his silver hair,
And be like him and he will then love me.

"""
predicted_emotions_df = predict_emotion(new_poem)

# Display the predicted emotion for each stanza
for stanza in predicted_emotions_df['Stanza'].unique():
    stanza_df = predicted_emotions_df.loc[predicted_emotions_df['Stanza'] == stanza]
    emotion = stanza_df.iloc[0]['Emotion']
    print(f"Stanza:\n{stanza}\nEmotion: {emotion}\n")


Accuracy: 0.37879767291531996
Classification Report:
               precision    recall  f1-score   support

       anger       0.46      0.19      0.27       159
     courage       0.54      0.30      0.38       152
        fear       0.00      0.00      0.00        64
        hate       0.00      0.00      0.00        41
         joy       0.43      0.28      0.34       292
        love       0.37      0.67      0.48       342
       peace       0.53      0.17      0.26       134
         sad       0.32      0.52      0.40       330
    surprise       1.00      0.03      0.06        33

    accuracy                           0.38      1547
   macro avg       0.41      0.24      0.24      1547
weighted avg       0.40      0.38      0.34      1547

Stanza:

My mother bore me in the southern wild,
And I am black, but O!my soul is white;
White as an angel is the English child: 
But I am black as if bereav'd of light.
Emotion: sad

Stanza:
My mother taught me underneath a tree 
And sittin

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv('PERC_mendelly.csv')

# Split poem into stanzas
df['Stanza'] = df['Poem'].str.split('\n\n')

# Split dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Vectorize stanzas
vectorizer = CountVectorizer()
train_X = vectorizer.fit_transform(train_df['Stanza'].apply(lambda x: '\n'.join(x)))
test_X = vectorizer.transform(test_df['Stanza'].apply(lambda x: '\n'.join(x)))

# Define target variable
target_var = 'Emotion'
train_y = train_df[target_var]
test_y = test_df[target_var]

# Train logistic regression model
clf = LogisticRegression(random_state=42)
clf.fit(train_X, train_y)

# Predict emotions for test set
pred_y = clf.predict(test_X)

# Print accuracy score
accuracy = accuracy_score(test_y, pred_y)
print(f'Accuracy score: {accuracy:.2f}')

# Predict emotions for each stanza in input poem
input_poem = """
My mother bore me in the southern wild,
And I am black, but O!my soul is white;
White as an angel is the English child: 
But I am black as if bereav'd of light.

My mother taught me underneath a tree 
And sitting down before the heat of day,
She took me on her lap and kissed me,
And pointing to the east began to say. 

Look on the rising sun: there God does live 
And gives his light, and gives his heat away. 
And flowers and trees and beasts and men receive
Comfort in morning joy in the noonday.

And we are put on earth a little space,
That we may learn to bear the beams of love, 
And these black bodies and this sun-burnt face
Is but a cloud, and like a shady grove.

For when our souls have learn'd the heat to bear 
The cloud will vanish we shall hear his voice. 
Saying: come out from the grove my love & care,
And round my golden tent like lambs rejoice.

Thus did my mother say and kissed me, 
And thus I say to little English boy. 
When I from black and he from white cloud free,
And round the tent of God like lambs we joy: 

Ill shade him from the heat till he can bear, 
To lean in joy upon our fathers knee. 
And then I'll stand and stroke his silver hair,
And be like him and he will then love me.
"""

input_stanzas = input_poem.split('\n\n')
input_X = vectorizer.transform([input_stanzas])
input_pred_y = clf.predict(input_X)
print(f'Emotions for input poem: {input_pred_y}')


Accuracy score: 0.43


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AttributeError: 'list' object has no attribute 'lower'